# GANs
GANs are a type of deep learning model used for generating data similar to a given dataset.

Applications:

Image Generation (e.g., creating realistic human faces).

Data Augmentation for training AI models.

Style Transfer (e.g., turning sketches into photos).

Video and Music Generation.


# GAN Architecture
GANs consist of two neural networks:

Generator (G):
Creates fake data to fool the discriminator.

Discriminator (D):
Classifies data as real or fake.

They work in a game-like setting (adversarial process):

The generator tries to trick the discriminator.

The discriminator learns to catch fakes.

Both improve over time.

### Key Concepts in GANs

Loss Functions:

Discriminator: Minimize binary cross-entropy loss to distinguish real and fake.

Generator: Maximize probability of generating realistic data.
Mode Collapse:

The generator might produce repetitive outputs.

Solution: Use Wasserstein GAN (WGAN) or Progressive GANs.
Evaluation Metrics:

FID Score (Fréchet Inception Distance): Measures similarity between generated and real data.
Advanced Variants:

DCGAN (Deep Convolutional GAN): Convolutional layers for images.

CycleGAN: Style transfer without paired data.

StyleGAN: High-quality image generation.

# Example Task: Generate Handwritten Digits (MNIST)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)


#### Define Generator and Discriminator

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


#### Optimizer

In [ ]:
generator = Generator(input_dim=100)
discriminator = Discriminator()

optimizer_g = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002)

loss_fn = nn.BCELoss()


#### Train the Gan

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    for batch in dataloader:
        # Prepare real and fake data
        real_images = batch[0].view(-1, 784)
        real_labels = torch.ones(real_images.size(0), 1)
        fake_labels = torch.zeros(real_images.size(0), 1)

        # Train Discriminator
        outputs = discriminator(real_images)
        d_loss_real = loss_fn(outputs, real_labels)

        noise = torch.randn(real_images.size(0), 100)
        fake_images = generator(noise)
        outputs = discriminator(fake_images.detach())
        d_loss_fake = loss_fn(outputs, fake_labels)

        d_loss = d_loss_real + d_loss_fake
        optimizer_d.zero_grad()
        d_loss.backward()
        optimizer_d.step()

        # Train Generator
        outputs = discriminator(fake_images)
        g_loss = loss_fn(outputs, real_labels)
        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()

    print(f"Epoch {epoch+1}/{num_epochs}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


#### Evaluate the GAN

In [ ]:
import matplotlib.pyplot as plt

# Generate fake images
with torch.no_grad():
    noise = torch.randn(64, 100)
    generated_images = generator(noise).view(-1, 1, 28, 28)

# Plot images
grid = torchvision.utils.make_grid(generated_images, nrow=8, normalize=True)
plt.figure(figsize=(10,10))
plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
plt.show()


In [1]:
from transformers import CarGANModel, CarGANTokenizer

# Load the CarGAN model and tokenizer
model = CarGANModel.from_pretrained("cargan")
tokenizer = CarGANTokenizer.from_pretrained("cargan")

# Define the text prompt
prompt = "A sleek, red sports car with a spoiler"

# Tokenize the prompt
inputs = tokenizer.encode_plus(prompt, return_tensors="pt")

# Generate the image
outputs = model.generate(inputs)

# Save the image to a file
image = outputs[0].permute(1, 2, 0).numpy()
import matplotlib.pyplot as plt
plt.imshow(image)
plt.savefig("car_image.png")

/home/vai/Desktop/chat_bot_experiment_one/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'CarGANModel' from 'transformers' (/home/vai/Desktop/chat_bot_experiment_one/myenv/lib/python3.12/site-packages/transformers/__init__.py)